# Data exploration

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import re
import pandas as pd

In [ ]:
filename = "data/Mg22_alphaalpha_digiSim.h5"

f = h5py.File(filename, "r")
# List all groups
#print("All keys: %s" % f.keys())
print("Total number of events:", len(f.keys()))
a_group_key = list(f.keys())[0]

print("Extracting data for key:", a_group_key)
# Get the data
data_0 = list(f[a_group_key])
#print(f.keys()) 

list_data = []
dict_data = {}
y = []

for i, key in enumerate(f.keys()):
    #if i > 10:
    #    break
    #print(key)
    re_m = re.match("Event_\[(\d*)\]", key)
    event = int(re_m.groups()[0])
    #print("Event:", event)

    tmp = np.asarray(list(f[key]))        
    dict_data[event] = tmp         
    list_data.append(tmp)
    y.append(event % 2)
    
    
y = np.asarray(y)[:,np.newaxis]

In [ ]:
hf = h5py.File(filename, "r")
hf.get('/get')

### Data structure

In [ ]:
display(pd.DataFrame(gi.get_event_by_index(hf, 1204)))

### Length of events

In [ ]:
length = []
for key in hf.keys():
    length.append(len(hf[key]))
length = np.asarray(length)

In [ ]:
plt.figure()
plt.hist(length, bins=100)
plt.xlabel("Length (items)")
plt.show()

## Visualisation

In [ ]:
event_i = 8

fig = plt.figure(figsize=(12,6))
ax = plt.subplot(131)
sc = plt.scatter(dict_data[event_i]["x"], dict_data[event_i]["y"], c=dict_data[event_i]["A"], cmap='inferno')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("XY projection")

ax = plt.subplot(132)
sc = plt.scatter(dict_data[event_i]["x"], dict_data[event_i]["z"], c=dict_data[event_i]["A"], cmap='inferno')
ax.set_xlabel("x")
ax.set_ylabel("z")
ax.set_title("XZ projection")

ax = plt.subplot(133)
sc = plt.scatter(dict_data[event_i]["y"], dict_data[event_i]["z"], c=dict_data[event_i]["A"], cmap='inferno')
ax.set_xlabel("y")
ax.set_ylabel("z")
ax.set_title("YZ projection")

cbar = fig.colorbar(sc, orientation='vertical')    

plt.show()

In [ ]:
n_rows, n_cols = 4, 4

fig = plt.figure(figsize=(12,12))

for i in range(n_rows*n_cols):
    ax = plt.subplot(n_rows, n_cols, i+1)
    sc = plt.scatter(dict_data[i]["x"], dict_data[i]["y"], c=dict_data[i]["A"], cmap='inferno')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("Event: {}".format(i))

plt.tight_layout()
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
event_i = 13

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(dict_data[event_i]["x"], dict_data[event_i]["y"],dict_data[event_i]["A"])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("XYZ")
plt.show()

## Discretization

In [ ]:
import data_discretization as dd
import generate_images as gi
%load_ext autoreload
%autoreload 1
%aimport data_discretization
%aimport generate_images

In [ ]:
gi.real_labeled("xy", "data/", "output/", "test_")

In [ ]:
import os
from generate_images import *

def _l(a):
    return 0 if a == 0 else math.log10(a)


In [ ]:
projection, data_dir, save_path, prefix = "xy", "data/", "output/", "test_"
print('Processing data...')
data = []

filename = os.path.join(data_dir, DATA_FILE)
h5_file = h5py.File(filename, "r")

for key in h5_file.keys():
    #event = events[str(evt_id)]
    #xyzs = event.xyzs(peaks_only=True, drift_vel=5.2, clock=12.5, return_pads=False,
    #                  baseline_correction=False,
    #                  cg_times=False)
    xyzs = np.asarray(pd.DataFrame(h5_file[key][:]))
    if xyzs.shape[0] > 0:
        data.append([xyzs, get_label(key)])
    else:
        print("WARNING,", key, "has no pads firing. Removing event ...")

log = np.vectorize(_l)

print()

for event in data:
    event[0][:, CHARGE_COL] = log(event[0][:, CHARGE_COL])

# Shuffle data
data = shuffle(data)

# Split into train and test sets
partition = int(len(data) * 0.8)
train = data[:partition]
test = data[partition:]

# Normalize
max_charge = np.array(list(map(lambda x: x[0][:, CHARGE_COL].max(), train))).max()

for e in train:
    for point in e[0]:
        point[CHARGE_COL] = point[CHARGE_COL] / max_charge

for e in test:
    for point in e[0]:
        point[CHARGE_COL] = point[CHARGE_COL] / max_charge

In [ ]:
#matplotlib.use('QT4Agg')
%matplotlib inline

In [ ]:
print('Making images...')

# Make train numpy sets
train_features = np.empty((len(train), 128, 128, 3), dtype=np.uint8)
train_targets = np.empty((len(train),), dtype=np.uint8)

for i, event in enumerate(train):
    if i > 5:
        break
    e = event[0]
    if projection == 'zy':
        x = e[:, Z_COL].flatten()
        z = e[:, Y_COL].flatten()
        c = e[:, CHARGE_COL].flatten()
    elif projection == 'xy':
        x = e[:, X_COL].flatten()
        z = e[:, Y_COL].flatten()
        c = e[:, CHARGE_COL].flatten()
    else:
        raise ValueError('Invalid projection value.')
    fig = plt.figure(figsize=(1, 1), dpi=128)
    if projection == 'zy':
        plt.xlim(0.0, 1250.0)
    elif projection == 'xy':
        plt.xlim(-275.0, 275.0)
    plt.ylim((-275.0, 275.0))
    #plt.axis('off')
    plt.scatter(x, z, s=0.6, c=c, cmap='Greys')
    fig.canvas.draw()
    data = np.array(fig.canvas.renderer._renderer, dtype=np.uint8)
    data = np.delete(data, 3, axis=2)
    train_features[i] = data
    train_targets[i] = event[1]
    plt.show()
    plt.close()

In [ ]:
data.shape

In [ ]:
d = {1: 100, 2: 200, 3: 300}
list(d.values())

In [ ]:
# Make test numpy sets
test_features = np.empty((len(test), 128, 128, 3), dtype=np.uint8)
test_targets = np.empty((len(test),), dtype=np.uint8)

for i, event in enumerate(test):
    e = event[0]
    if projection == 'zy':
        x = e[:, 2].flatten()
        z = e[:, 1].flatten()
        c = e[:, 3].flatten()
    elif projection == 'xy':
        x = e[:, 0].flatten()
        z = e[:, 1].flatten()
        c = e[:, 3].flatten()
    else:
        raise ValueError('Invalid projection value.')
    fig = plt.figure(figsize=(1, 1), dpi=128)
    if projection == 'zy':
        plt.xlim(0.0, 1250.0)
    elif projection == 'xy':
        plt.xlim(-275.0, 275.0)
    plt.ylim((-275.0, 275.0))
    plt.axis('off')
    plt.scatter(x, z, s=0.6, c=c, cmap='Greys')
    fig.canvas.draw()
    data = np.array(fig.canvas.renderer._renderer, dtype=np.uint8)
    data = np.delete(data, 3, axis=2)
    test_features[i] = data
    test_targets[i] = event[1]
    plt.close()

print('Saving file...')

if not os.path.exists(save_path):
    os.makedirs(save_path)

filename = os.path.join(save_path, prefix + 'images.h5')

# Save to HDF5
h5 = h5py.File(filename, 'w')
h5.create_dataset('train_features', data=train_features)
h5.create_dataset('train_targets', data=train_targets)
h5.create_dataset('test_features', data=test_features)
h5.create_dataset('test_targets', data=test_targets)
h5.create_dataset('max_charge', data=np.array([max_charge]))
h5.close()

In [ ]:
np.delete()

In [ ]:
np.asarray(pd.DataFrame(h5_file[key][:])).shape

In [ ]:
event[1]

In [ ]:
dict_data = gi.read_and_label_data('data/').values()

In [ ]:
list(dict_data)

In [ ]:
gi.generate_image_data_set("xy", "data/", "output/", "test_")

In [ ]:
event = 8
dict_data = gi.read_and_label_data("data/")
data = list(dict_data[8]) #from dict to list
print("Shape:\n\tdata:", len(data))
data, max_charge = gi.transform_normalize_data(data)